In [5]:
import os
from Bio import SeqIO 
import shutil

In [17]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
OUT_PATH_NUCMER = os.path.join(BASE_AA_PATH, 'nucmer_analysis')
OUT_PATH_ASSEMBLETICS = os.path.join(BASE_AA_PATH, 'Assembletics')
MUMMER_PATH_PREFIX = '/home/benjamin/anaconda3/bin/'






if not os.path.isdir(OUT_PATH_NUCMER):
    os.mkdir(OUT_PATH_NUCMER)
if not os.path.isdir(OUT_PATH_ASSEMBLETICS):
    os.mkdir(OUT_PATH_ASSEMBLETICS)

    


In [6]:
#Define your p and h genome and move it into the allele analysis folder
genome_prefix  = 'Pst_104E_v12'
p_genome = 'Pst_104E_v12_p_ctg'
h_genome = 'Pst_104E_v12_h_ctg'
genome_file_suffix = '.genome_file'
for x in (x + '.fa' for x in [p_genome, h_genome]):
    shutil.copy2(BASE_A_PATH+'/'+x, OUT_PATH_NUCMER)
    shutil.copy2(BASE_A_PATH+'/'+x, OUT_PATH_ASSEMBLETICS)

In [7]:
#define the scripts to generate
bash_script_q= genome_prefix+"_ph_ctg_qmapping.sh"
bash_script_g=genome_prefix+"_ph_ctg_gmapping.sh"
bash_script_nucmer_assemlytics = genome_prefix +"_ncumer_assemblyticsmapping.sh"


In [ ]:
outfq = open(os.path.join(OUT_PATH_NUCMER, bash_script_q), 'w')
outfq.write('#!/bin/bash\n')
outfg = open(os.path.join(OUT_PATH_NUCMER,bash_script_g), 'w')
outfg.write('#!/bin/bash\n')#parsing out P and corresponding h contigs and writing a short ncumer script that aligns them against each other
outfna = open(os.path.join(OUT_PATH_ASSEMBLETICS,bash_script_nucmer_assemlytics), 'w')
outfna.write('#!/bin/bash\n')

for pseq_record in SeqIO.parse(OUT_PATH_ASSEMBLETICS+'/'+genome_prefix+'_p_ctg.fa', 'fasta'):
    p_acontigs = []
    p_contig = pseq_record.id.split("_")[0]+"_"+pseq_record.id.split("_")[1]
    suffix = genome_prefix+"_"+p_contig+"_php"
    p_file = genome_prefix+"_"+p_contig+'.fa'
    SeqIO.write(pseq_record, OUT_PATH_ASSEMBLETICS+'/'+ p_file, 'fasta')
    SeqIO.write(pseq_record,OUT_PATH_NUCMER+'/'+ p_file, 'fasta')
    for aseq_record in SeqIO.parse(OUT_PATH_ASSEMBLETICS+'/'+genome_prefix+'_h_ctg.fa', 'fasta'):
        if aseq_record.id.split("_")[1]  == pseq_record.id.split("_")[1]:
            p_acontigs.append(aseq_record)
    a_file = genome_prefix +"_"+pseq_record.id.split("_")[0]+"_"+pseq_record.id.split("_")[1]+'_h_ctgs.fa'
    #if we have alternative contigs safe those too
    if p_acontigs != []:
        SeqIO.write(p_acontigs, OUT_PATH_ASSEMBLETICS+'/'+  a_file, 'fasta')
        SeqIO.write(p_acontigs, OUT_PATH_NUCMER+'/'+  a_file, 'fasta')
        outfq.write(MUMMER_PATH_PREFIX +'/nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfq.write(MUMMER_PATH_PREFIX +'/delta-filter -q '+'out.delta'+" > "+suffix+"_qfiltered.delta\n")
        outfq.write(MUMMER_PATH_PREFIX +'/show-coords -T '+suffix+"_qfiltered.delta > "+suffix+".qcoords\n")
        outfq.write(MUMMER_PATH_PREFIX +'/mummerplot -p '+suffix+'_qfiltered --png '+suffix+"_qfiltered.delta\n")
        outfq.write(MUMMER_PATH_PREFIX +'/mummerplot -c -p '+suffix+'_qfiltered_cov --png '+suffix+"_qfiltered.delta\n")
        #for g_file bash script
        outfg.write(MUMMER_PATH_PREFIX +'/nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfg.write(MUMMER_PATH_PREFIX +'/delta-filter -g '+'out.delta'+" > "+suffix+"_gfiltered.delta\n")
        outfg.write(MUMMER_PATH_PREFIX +'/show-coords -T '+suffix+"_gfiltered.delta > "+suffix+".gcoords\n")
        outfg.write(MUMMER_PATH_PREFIX +'/mummerplot -p '+suffix+'_gfiltered --png '+suffix+"_gfiltered.delta\n")
        outfg.write(MUMMER_PATH_PREFIX +'/mummerplot -c -p  '+suffix+'_gfiltered_cov --png '+suffix+"_gfiltered.delta\n")
        #for nucmer assemblytics out
        outfna.write(MUMMER_PATH_PREFIX +'/nucmer -maxmatch -l 100 -c 500 '+p_file+' '+a_file+' -prefix ' + suffix +'\n')
outfna.close()    
outfq.close()
outfg.close()  

In [ ]:
!mkdir assemblytics
!cp * assemblytics

In [ ]:
outfnarun = open('Pst_E104_v1_assemblytics.sh', 'w')
outfnarun.write('#!/bin/bash\n')

files = !ls assemblytics/
delta_files = [x for x in files if 'delta' in x]
assemblytics_path = '/Users/ben/scripts/Assemblytics'
outfnarun.write('export PATH="%s":$PATH\n'% assemblytics_path)
outfnarun.write('#Assemblytics delta_file output_prefix unique_anchor_length path_to_R_scripts\n')
for delta in delta_files:
    delta_10kbp = delta[:-6] + '_10kbp'
    delta_50kbp = delta[:-6] + '_50kbp'
    outfnarun.write("mkdir %s\ncp %s %s\ncd %s\n" % (delta[:-6], delta, delta[:-6], delta[:-6]))
    outfnarun.write("Assemblytics %s %s 10000 %s\n" % (delta, delta_10kbp, assemblytics_path) )
    outfnarun.write("Assemblytics %s %s 50000 %s\ncd ..\n" % (delta, delta_50kbp, assemblytics_path) )

outfnarun.close()
!mv Pst_E104_v1_assemblytics.sh assemblytics/

In [ ]:
delta_files[0][:-6]

In [ ]:
for pseq_record in SeqIO.parse('Pst_E104_v1__pcontig_000_h_ctgs.fa', 'fasta'):
    print(pseq_record.id +', %i'%len(pseq_record.seq))